In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [ ]:
loader = TextLoader("./data/state_of_the_union.txt")
loaded_document = loader.load()

In [ ]:
loaded_document

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [ ]:
len(chunks_of_text)

In [ ]:
embeddings = OpenAIEmbeddings()

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [ ]:
vector_db

In [ ]:
retriever = vector_db.as_retriever()

In [ ]:
response = retriever.invoke("what did he say about ketanji brown jackson?")

In [ ]:
response

In [ ]:
len(response)

In [ ]:
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [ ]:
response = retriever.invoke("what did he say about ketanji brown jackson?")

In [ ]:
response

## Simple use with LCEL

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
template = """Answer the question based only on the following context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()


In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [ ]:
response = chain.invoke("what did he say about ketanji brown jackson?")

In [ ]:
response

In [ ]:
type(response)